In [7]:
import pandas as pd


# debugger
def debug(print_statement):
    print(print_statement)


# voorbeelddata
testdata = [
    {
        "Cursuscode": "KP22A",
        "Startdatum": pd.to_datetime("2022-04-01"),
        "Einddatum": pd.to_datetime("2026-02-15"),
        "Omzet": 500_000,
        2022: 100_000,
        2023: 150_000,
    }
]
df = pd.DataFrame(testdata)

"""_summary_
Neemt cursusinformatie en geeft een dataframe waarin de omzet lineair per week verdeeld wordt. Houdt daarbij rekening met gegeven jaaromzetten. Als een jaaromzet gegeven is, wordt die omzet binnen het jaar lineair verdeeld.
"""


def calc_omzet_per_week(row) -> pd.DataFrame:
    # 1. maak een dict met omzetweken in het jaar

    # dit zijn alle omzetweken in totaal (als datum datatype)
    daterange = pd.date_range(
        row["Startdatum"], row["Einddatum"], freq="W", inclusive="both"
    )
    vakantieweken = [1, 8, 18, 19, 29, 30, 31, 32, 33, 34, 51, 52]
    weken_zonder_vakantie = []
    lesweken_per_jaar = {}

    # alle weken met een jaartal dat voorkomt in de gegeven data worden apart genomen.
    # de rest van de weken wordt onder de sleutel 'rest' opgenomen.
    for week in daterange:
        if week.isocalendar().week in vakantieweken:
            continue
        sleutel = "resterend"
        if week.year in row.index:
            sleutel = week.year
        if sleutel in lesweken_per_jaar:
            lesweken_per_jaar[sleutel].append(week)
        else:
            lesweken_per_jaar[sleutel] = [week]

    df_result = pd.DataFrame(columns=weken_zonder_vakantie)

    # we gaan per jaar de omzet per week berekenen
    # als voor een jaar een omzetbedrag gegeven is, wordt die omzet lineair
    # verdeeld in het jaar
    restomzet = row["Omzet"]  # de totaal resterende, te verdelen omzet
    for jaar, weken in lesweken_per_jaar.items():
        jaaromzet = 0
        weekomzet = 0
        aantal_lesweken = len(lesweken_per_jaar[jaar])
        if jaar in row.index:  # is er een gegeven omzet voor dit jaar?
            jaaromzet = row[jaar]
            weekomzet = jaaromzet / aantal_lesweken
            restomzet -= jaaromzet
            debug(
                f"Setting jaaromzet to {jaaromzet} for {jaar} met weekomzet {weekomzet} in {aantal_lesweken}. Restomzet: {restomzet}"
            )
        else:
            weekomzet = restomzet / aantal_lesweken
            debug(
                f"Geen jaaromzet gevonden voor {jaar}. Resterende omzet is {restomzet} in {aantal_lesweken} weken. Weekomzet is {weekomzet}"
            )
            # wat is de weekomzet voor dit jaar?
        for week in weken:
            newrow = {"Jaar": week.year, "Week": week, "Omzet": weekomzet}
            df_result = pd.concat(
                [df_result, pd.DataFrame.from_dict([newrow])], ignore_index=True
            )
    df_result.insert(0, "Cursuscode", row["Cursuscode"])

    return df_result


result = calc_omzet_per_week(df.iloc[0, :])

Setting jaaromzet to 100000 for 2022 met weekomzet 3333.3333333333335 in 30. Restomzet: 400000
Setting jaaromzet to 150000 for 2023 met weekomzet 3750.0 in 40. Restomzet: 250000
Geen jaaromzet gevonden voor resterend. Resterende omzet is 250000 in 86 weken. Weekomzet is 2906.9767441860463


In [8]:
result.transpose()

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
Cursuscode,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,...,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A,KP22A
Jaar,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2025,2025,2025,2025,2026,2026,2026,2026,2026,2026
Week,2022-04-03 00:00:00,2022-04-10 00:00:00,2022-04-17 00:00:00,2022-04-24 00:00:00,2022-05-01 00:00:00,2022-05-22 00:00:00,2022-05-29 00:00:00,2022-06-05 00:00:00,2022-06-12 00:00:00,2022-06-19 00:00:00,...,2025-11-23 00:00:00,2025-11-30 00:00:00,2025-12-07 00:00:00,2025-12-14 00:00:00,2026-01-11 00:00:00,2026-01-18 00:00:00,2026-01-25 00:00:00,2026-02-01 00:00:00,2026-02-08 00:00:00,2026-02-15 00:00:00
Omzet,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,3333.333333,...,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744,2906.976744


In [9]:
result_jaar = result[["Jaar", "Omzet"]].groupby("Jaar").sum()
result_jaar

,Omzet
Jaar,
2022,100000.000000
2023,150000.000000
2024,116279.069767
2025,116279.069767
2026,17441.860465
